## Tax_roll_impacts_dashboard

E. Quinn 5/22/2020

This notebook uses the tax roll information to build a single-family home property tax dashboard

## Import standard python datascience packages

In [ ]:
import math
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [ ]:
changes = pd.read_pickle(r'../tax_roll_changes.pkl')

In [ ]:
changes['044-011-418-0000']

In [ ]:
!pwd

### Add in FY2021 information from NE Reval and Town Manager

In [ ]:
ner = pd.read_csv('../eg12312019_priors.csv')
print(ner.columns)
print(ner.head())

nerd = ner.to_dict()

change2021 = {}

for key in nerd['ParcelID'].keys():
    parcel = nerd['ParcelID'][key].replace(' ','-')
    if (nerd['FY'][key]==2020):
        val = nerd['Total_value'][key]
        rate = 23.29
        if parcel in changes.keys():
            if 2020 in changes[parcel].keys():
                changes[parcel][2021] = {}
                try:
                    changes[parcel][2021]['address'] = changes[parcel][2020]['address']
                    changes[parcel][2021]['v2'] = val
                    changes[parcel][2021]['v1'] = changes[parcel][2020]['v2']
                    changes[parcel][2021]['r2'] = rate
                    changes[parcel][2021]['r1'] = changes[parcel][2020]['r2']
                    changes[parcel][2021]['t2'] = round(val*rate/1000.0,2)
                    changes[parcel][2021]['t1'] = changes[parcel][2020]['t2']

                    changes[parcel][2021]['delta_r'] = round(\
                        changes[parcel][2021]['r2'] - changes[parcel][2021]['r1'],2)
                    changes[parcel][2021]['delta_v'] = round(\
                        changes[parcel][2021]['v2'] - changes[parcel][2021]['v1'],2)
                    changes[parcel][2021]['delta_t'] = round(\
                        changes[parcel][2021]['t2'] - changes[parcel][2021]['t1'],2)
                    changes[parcel][2021]['delta_t_dr'] = (1/1000.0)*round(\
                        changes[parcel][2021]['v1']*changes[parcel][2021]['delta_r'],2)
                    changes[parcel][2021]['delta_t_dv'] = round(\
                        changes[parcel][2021]['r1']*changes[parcel][2021]['delta_v'],2)
                    changes[parcel][2021]['delta_t_drdv'] = (1/1000.)*round(\
                        changes[parcel][2021]['delta_r']*changes[parcel][2021]['delta_v'],2)
                except KeyError:
                    print(parcel,changes[parcel][2020])
        else: 
            continue

In [ ]:
changes['044-011-418-0000']

In [ ]:
sfd = {} 

for p in changes.keys():  
    if p not in sfd.keys():
        sfd[p] = {}
        sfd[p]['fy'] = []
        sfd[p]['val1'] = []
        sfd[p]['val2'] = []
        sfd[p]['rate1'] = []
        sfd[p]['rate2'] = []
        sfd[p]['tax1'] = []
        sfd[p]['tax2'] = []
        sfd[p]['address'] = ''
    for fy in changes[p].keys():
        if 'v1' in changes[p][fy].keys():
            sfd[p]['fy'].append(fy)
            sfd[p]['address'] = changes[p][fy]['address']
            sfd[p]['val1'].append(changes[p][fy]['v1'])
            sfd[p]['val2'].append(changes[p][fy]['v2'])
            sfd[p]['tax1'].append(changes[p][fy]['t1'])
            sfd[p]['tax2'].append(changes[p][fy]['t2'])
            sfd[p]['rate1'].append(changes[p][fy]['r1'])
            sfd[p]['rate2'].append(changes[p][fy]['r2'])

In [ ]:
sfd

In [ ]:
#tax rate history

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']
rate = sfd['044-011-418-0000']['rate2']


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%2.2f' % (x * 1e-0)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, rate)
ax.set_xlabel('Fiscal Year')

plt.show()


In [ ]:
#valuation history

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']
val = sfd['044-011-418-0000']['val2']


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%3.0fK' % (x * 1e-3)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, val)

plt.show()


In [ ]:
#property tax history

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']
tax = sfd['044-011-418-0000']['tax2']


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%6.0f' % (x * 1.0e-0)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, tax)

plt.show()


In [ ]:
#percent change in property tax

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']

pctchg = []

for i in np.arange(0,len(sfd['044-011-418-0000']['tax2'])):
    pctchg.append(-1.0 + sfd['044-011-418-0000']['tax2'][i]/sfd['044-011-418-0000']['tax1'][i])


def millions(x, pos):
    'The two args are the value and tick position'
    return '%3.1f' % (x * 1.0e+2)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, pctchg)
#plt.xticks(x, ('Bill', 'Fred', 'Mary', 'Sue'))
plt.show()


In [ ]:
#net change in property tax

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']

pctchg = []

for i in np.arange(0,len(sfd['044-011-418-0000']['tax2'])):
    pctchg.append(sfd['044-011-418-0000']['tax2'][i]-sfd['044-011-418-0000']['tax1'][i])


def millions(x, pos):
    'The two args are the value and tick position'
    return '%6.0f' % (x * 1.0e+0)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, pctchg)

plt.show()


In [ ]:
#change in tax bill due to tax rate change

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']

trchg = []

for i in np.arange(0,len(sfd['044-011-418-0000']['tax2'])):
    rate_delta = sfd['044-011-418-0000']['rate2'][i] - sfd['044-011-418-0000']['rate1'][i]
    trchg.append(sfd['044-011-418-0000']['val1'][i]*rate_delta/1000.0)


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%6.0f' % (x * 1.0e-0)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, trchg)

plt.show()


In [ ]:
#change in tax bill due to assessed value change

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']

asvchg = []

for i in np.arange(0,len(sfd['044-011-418-0000']['tax2'])):
    val_delta = sfd['044-011-418-0000']['val2'][i] - sfd['044-011-418-0000']['val1'][i]
    asvchg.append(sfd['044-011-418-0000']['rate1'][i]*val_delta/1000.0)


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%6.0f' % (x * 1.0e-0)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, asvchg)

plt.show()


In [ ]:
#change in tax bill due to interaction of tax rate and assessed value changes

from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np

fy = sfd['044-011-418-0000']['fy']

combchg = []

for i in np.arange(0,len(sfd['044-011-418-0000']['tax2'])):
    val_delta = sfd['044-011-418-0000']['val2'][i] - sfd['044-011-418-0000']['val1'][i]
    rate_delta = sfd['044-011-418-0000']['rate2'][i] - sfd['044-011-418-0000']['rate1'][i]
    combchg.append(rate_delta*val_delta/1000.0)


def millions(x, pos):
    'The two args are the value and tick position'
    return '$%6.0f' % (x * 1.0e-0)


formatter = FuncFormatter(millions)

fig, ax = plt.subplots()
ax.yaxis.set_major_formatter(formatter)
plt.bar(fy, combchg)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

p = '044-011-418-0000'

fy = sfd[p]['fy']

trchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    rate_delta = sfd[p]['rate2'][i] - sfd[p]['rate1'][i]
    trchg.append(sfd[p]['val1'][i]*rate_delta/1000.0)

asvchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    val_delta = sfd[p]['val2'][i] - sfd[p]['val1'][i]
    asvchg.append(sfd[p]['rate1'][i]*val_delta/1000.0)
    
combchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    val_delta = sfd[p]['val2'][i] - sfd[p]['val1'][i]
    rate_delta = sfd[p]['rate2'][i] - sfd[p]['rate1'][i]
    combchg.append(rate_delta*val_delta/1000.0)

#Calculate optimal width
width = np.min(np.diff(fy))/0.5

fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(fy-2*width,trchg,width,color='b',label='-Ymin')
ax.bar(fy-width,asvchg,width,color='r',label='Ymax')
ax.bar(fy,combchg,width,color='g',label='Ymax')
ax.set_xlabel('Test histogram')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

p = '044-011-418-0000'

fy = sfd[p]['fy']
val = sfd[p]['val2']
rate = sfd[p]['rate2']
address = sfd[p]['address']
tax = sfd[p]['tax2']

pctchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    pctchg.append(100.0*(-1.0 + sfd[p]['tax2'][i]/\
                  sfd[p]['tax1'][i]))
    
ratechg = []

for i in np.arange(0,len(sfd[p]['rate2'])):
    ratechg.append(100.0*(-1.0 + sfd[p]['rate2'][i]/\
                  sfd[p]['rate1'][i]))
    
netchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    netchg.append(sfd[p]['tax2'][i]-sfd[p]['tax1'][i])
    
trchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    rate_delta = sfd[p]['rate2'][i] - sfd[p]['rate1'][i]
    trchg.append(sfd[p]['val1'][i]*rate_delta/1000.0)
    
asvchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    val_delta = sfd[p]['val2'][i] - sfd[p]['val1'][i]
    asvchg.append(sfd[p]['rate1'][i]*val_delta/1000.0)
    
combchg = []

for i in np.arange(0,len(sfd[p]['tax2'])):
    val_delta = sfd[p]['val2'][i] - sfd[p]['val1'][i]
    rate_delta = sfd[p]['rate2'][i] - sfd[p]['rate1'][i]
    combchg.append(rate_delta*val_delta/1000.0)
    
valchg = []

for i in np.arange(0,len(sfd[p]['val2'])):
    valchg.append(100.0*(-1.0 + sfd[p]['val2'][i]/\
                  sfd[p]['val1'][i]))

fig, axs = plt.subplots(3, 3) #, constrained_layout=True)

axs[0,0].bar(fy, val)
axs[0,0].set_title('Assessed Value',fontsize=18)
axs[0,0].set_ylabel('Assessed Value',fontsize=14)
fig.suptitle(address, fontsize=24)
fig.set_size_inches(18.5, 18.5)

axs[0,1].bar(fy, rate)
axs[0,1].set_title('Tax Rate',fontsize=18)
axs[0,1].set_ylabel('Dollars per Thousand',fontsize=14)

axs[0,2].bar(fy, tax)
axs[0,2].set_title('Tax Paid',fontsize=18)
axs[0,2].set_ylabel('Dollars',fontsize=14)

axs[1,0].bar(fy, netchg)
axs[1,0].set_title('Change in Tax from Previous Year',fontsize=16)
axs[1,0].set_ylabel('Dollars',fontsize=14)

axs[1,1].bar(fy, ratechg)
axs[1,1].set_title('% Change in Tax Rate from Previous Year',fontsize=16)
axs[1,1].set_ylabel('Dollars',fontsize=14)

axs[1,2].bar(fy, valchg)
axs[1,2].set_title('% Change in Valuation from Previous Year',fontsize=16)
axs[1,2].set_ylabel('Dollars',fontsize=14)

axs[2,0].bar(fy, trchg)
axs[2,0].set_title('Effect of Rate Change on Tax',fontsize=18)
axs[2,0].set_ylabel('Dollars',fontsize=14)

axs[2,1].bar(fy, asvchg)
axs[2,1].set_title('Effect of Valuation Change on Tax',fontsize=18)
axs[2,1].set_ylabel('Dollars',fontsize=14)

axs[2,2].bar(fy, combchg)
axs[2,2].set_title('Rate/Valuation Interaction Effect on Tax',fontsize=18)
axs[2,2].set_ylabel('Dollars',fontsize=14)

fname = '../' + p + '.png'

fig.savefig(fname)

plt.show()